In [40]:
import pandas as pd
import numpy as np
dd = {'等级': list(range(1,5)), '组1':[122,63,2494,2912], '组2':[137,28,2577,2586], '组3':[134,25,2752,2511],
      '组4':[110,28,2506,2564],'组5':[121,37,2372,2753]}
df = pd.DataFrame(dd)
df

,等级,组1,组2,组3,组4,组5
0,1,122,137,134,110,121
1,2,63,28,25,28,37
2,3,2494,2577,2752,2506,2372
3,4,2912,2586,2511,2564,2753


In [66]:
#----------------------------------Ridit for multi-diversity
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.oneway import anova_generic
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.sandbox.stats.multicomp import MultiComparison
class RiditAnalysis:
    """
        默认数据为DataFrame格式(dataFrame);
        默认等级数据在第一列;
        默认alpha = 0.05;
    """
    def __init__(self, dataFrame, alpha=0.05):
            self.df = dataFrame
            self.alpha = alpha
    
    def multi_diversity(self):
        """
            多样本Ridit分析
            该版本根据statsmodels库的DescrStatsW类计算出方差分析所需的值用anova_generic方法进行方差分析，
            最后返回两个元素：
                1.self.df1:类型：DataFrame,包含--可信区间、平均秩次、标准差
                2.anova_result:类型：HolderTuple, 包含方差分析结果
            
        """
        self.df['标准组'] = pd.Series(0)  #初始化标准组
        nobs1 = self.df.sum() #求每组例数之和
        self.df['标准组']=st_group = self.df.sum(axis=1) - self.df.iloc[:,0] #求出标准组的值
        R = (np.cumsum(st_group) - st_group / 2) / st_group.sum() #求出R值
        self.df1 = pd.DataFrame()
        for index,colms in self.df.iloc[:,1:].iteritems():  #由于第一列为等级值，故用df.iloc[:,1:]跳过
            weighted_data = sm.stats.DescrStatsW(R,weights=colms)  #对数据加权
            ci=weighted_data.tconfint_mean(alpha=self.alpha)  #求出每组置信区间
            mean = weighted_data.mean  #求每组均值
            std = weighted_data.std  #求每组标准差
            df1_r = pd.DataFrame({'组别':[index], '可信区间':[ci], '平均秩次':mean, '标准差':std}) 
            self.df1 = self.df1.append(df1_r, ignore_index=True)  #将每组数据存储在新DataFrame中
        self.df2 = self.df1.copy()   #复制一个df1,因为下面需要对df1进行切片,后面两两比较要用
        self.df1.drop(len(self.df1)-1,inplace=True) #丢弃最后一行数据--标准组,为后面ANOVA做准备
        nobs1 = np.asarray(nobs1.iloc[1:len(nobs1)-1]) #对例数数组进行切片
        vars = self.df1['标准差']**2 #每组方差
        anova_result = anova_generic(self.df1['平均秩次'], vars, nobs1, use_var='equal') #ANOVA
        return self.df2, anova_result
    
    def weighted(slef,data, weights):
        """
            加权函数
            默认数据格式:DataFrame
            如果weights为n维，结果将根据其维度分组；若为1维，组别将全为1
            结果返回一个DataFrame
            参数：
                data:待加权数据、类型：Series
                weights:权值(任意维度)、类型：DataFrame
        """
        wes = weights.values.T  #获取DataFrame的元素值
        li1, li2 = [],[]
        if( len(wes) > 1):
            for id1, i in enumerate(data):
                for id2, j in enumerate(wes):
                    for id3, k in enumerate(j):
                        if(id1 == id3):
                            li2 += [i]*k
                            li1 += [id2+1]*k
        elif( len(wes) == 1):
            wes = wes.ravel() #降维
            for id1, i in enumerate(data):
                for id2, j in enumerate(wes):
                    if(id1 == id2):
                        li2 += [i]*j
            li1 = [1]*wes.sum()
        else:
            print('null weights')
            return 0
        dict1 = {'data':li2, '组别':li1}
        df = pd.DataFrame(dict1)
        return df 
    
    def ana(self):
        """
        
        """
        st_group = self.df.sum(axis=1) - self.df.iloc[:,0] #求出标准组的值
        R = (np.cumsum(st_group) - st_group / 2) / st_group.sum() #求出R值
        df_weighted = self.weighted(R, df.iloc[:,1:])  #加权数据
        model = ols('data~C(组别)', df_weighted).fit()
        anova_result = anova_lm(model)
        #如果方差分析P值小于alpha，则进行两两比较
        if (anova_result.values[0][-1] < self.alpha):
            tukeyHSD = MultiComparison(df_weighted['data'], df_weighted['组别']).tukeyhsd()
        else:
            tukeyHSD = 0
        return anova_result, tukeyHSD
    
    def visual_ci(self): #可视化
        err = [x[1] for x in self.df1['可信区间']] - self.df1['平均秩次']
        fig, ax = plt.subplots()  #创建画布
        ax.errorbar(self.df1.index+1, self.df1['平均秩次'], err, fmt='o', linewidth=2, capsize=6) #画可信区间
        plt.axhline(y=0.5,c='red') #画y=0.5这条直线
        plt.xlabel('group'), plt.ylabel('ci')  #坐标轴标签
        plt.show()
        return 0
    
rd1 = RiditAnalysis(df)
# df22, anre = rd1.multi_diversity()


In [68]:
ar, tk = rd1.ana()
tk.summary()

group1,group2,meandiff,p-adj,lower,upper,reject
1,2,-0.0168,0.005,-0.03,-0.0035,True
1,3,-0.0271,0.001,-0.0403,-0.014,True
1,4,-0.0122,0.0886,-0.0255,0.0011,False
1,5,0.0009,0.9,-0.0124,0.0141,False
2,3,-0.0104,0.2098,-0.0237,0.003,False
2,4,0.0045,0.886,-0.0089,0.018,False
2,5,0.0176,0.0031,0.0042,0.0311,True
3,4,0.0149,0.0206,0.0015,0.0283,True
3,5,0.028,0.001,0.0147,0.0414,True
4,5,0.0131,0.0616,-0.0004,0.0266,False


In [9]:
df22

,组别,可信区间,平均秩次,标准差
0,组1,"(0.5043343162426497, 0.5176919695601894)",0.511013,0.254721
1,组2,"(0.4874438602139169, 0.5010713138935539)",0.494258,0.253676
2,组3,"(0.47716069020049007, 0.4905938265027004)",0.483877,0.252256
3,组4,"(0.4919222944315486, 0.5056274163706941)",0.498775,0.252231
4,组5,"(0.5050486798446963, 0.5187329413949298)",0.511891,0.253655
5,标准组,"(0.496966070107062, 0.5030339298929379)",0.500000,0.253546


In [4]:
anre

<class 'statsmodels.stats.base.HolderTuple'>
statistic = 11.757361551708419
pvalue = 1.5322296190620433e-09
df = (4.0, 26827.0)
df_num = 4.0
df_denom = 26827.0
nobs_t = 26832.0
n_groups = 5
means = 0    0.511013
1    0.494258
2    0.483877
3    0.498775
4    0.511891
Name: 平均秩次, dtype: float64
nobs = [5591. 5328. 5422. 5208. 5283.]
vars_ = 0    0.064883
1    0.064351
2    0.063633
3    0.063620
4    0.064341
Name: 标准差, dtype: float64
use_var = equal
welch_correction = True
tuple = (11.757361551708419, 1.5322296190620433e-09)